In [ ]:
from openai import OpenAI
from docx import Document
import pandas as pd
import os
import json
import re
import time

# -----------------------------
# LLM Client Setup
# -----------------------------
client = OpenAI(
    base_url="http://10.173.91.239:1234/v1",
    api_key="lm-studio"
)

model_name = "google/gemma-3-4b"

# -----------------------------
# File Paths
# -----------------------------
INPUT_FILE = "750-tweets-without answer.xlsx"
OUTPUT_FILE = "Gemma3_old_prompt1.xlsx"


# -----------------------------
# JSON Schema for classification
# -----------------------------
classification_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "AgeismClassification",
        "schema": {
            "type": "object",
            "properties": {
                "classification": {
                    "type": "string",
                    "enum": ["Not relevant", "Ageist", "Age-positive"]
                }
            },
            "required": ["classification"]
        },
    }
}


# -----------------------------
# Load input Excel file
# -----------------------------
df = pd.read_excel(INPUT_FILE, dtype=str)
df["classification"] = ""   # empty column to fill later

# Load keyword list
keywords_doc = Document("Keywords list_final.docx")
keywords_text = "\n".join([p.text for p in keywords_doc.paragraphs])

# -----------------------------
# Loop through tweets
# -----------------------------
for i, row in df.iterrows():
    start_time = time.time()
    text = row["text"]  # make sure your tweet text column is called "text"

    prompt_text = f"""
    You are an assistant analyzing social media posts to determine whether they contain age-related content. 
    First decide whether the tweet is truly about an older adult—someone in their later decades of life. 
    Do not treat a tweet as aging-related when words like “old,” “senior,” “elder,” “granny,” “antique,” “ancient,” “pops,” or references to nursing homes describe objects, metaphors, younger people, or unrelated contexts such as the Old Testament, an old car, high-school seniors, church elders, Elder Scrolls, granny smith apples, or nostalgic “old times.” Similarly, “middle-aged” does not count as older adulthood, and phrases like “old people” should only be considered aging-related if the context clearly refers specifically to elderly individuals. Tweets that merely mention an older person without expressing a positive or negative judgment should be treated as neutral and therefore not relevant.
    If the tweet is aging-related, classify it as ageist when it portrays an older adult negatively, mockingly, or stereotypically, including references to frailty, slowness, incompetence, dependency, forgetfulness, dementia, or cognitive decline. Casual humor, sarcasm, or insults directed at older adults should also be treated as ageist. Tweets that invoke dementia, Alzheimer’s disease, or memory loss in a derogatory or belittling way should always be coded as ageist because they reinforce negative age-based stereotypes.
    Classify a tweet as age-positive when it expresses affection, respect, admiration, gratitude, pride, or positive remembrance of an older adult. Examples include praising older adults for their wisdom, resilience, accomplishments, character, or contributions, as well as tweets expressing love for or missing an older relative, even one who has passed away. Any positive sentiment toward an older adult should be considered age-positive.
    If the tweet is not related to older adults at all, or uses age-related wording only metaphorically or about objects, situations, or non-human entities, classify it as not relevant. Likewise, if a tweet mentions an older adult but does so in a purely descriptive or neutral way without any positive or negative portrayal, it should also be classified as not relevant. 
    Your output should consist of only one classification label for each tweet: “ageist,” “age-positive,” or “not relevant.”
    keyword list: {keywords_text}
    Post: {text}
    """

    # LLM call
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": [{"type": "text", "text": prompt_text.strip()}]}],
        response_format=classification_schema,
        )

    raw = response.choices[0].message.content.strip()
    #print(f"🏷️ LLM output raw: {raw}")

    # Parse JSON safely
    try:
        parsed = json.loads(raw)
        classification = parsed.get("classification", "Not relevant")
    except Exception:
        classification = "Not relevant"

    df.at[i, "classification"] = classification

    print(f"\n📄 Tweet {i+1} of {len(df)}")
    print(f"🏷️ Classification: {classification}")
    #print(f"⏱️ Runtime: {round(time.time() - start_time, 2)} seconds")

# -----------------------------
# Save output
# -----------------------------
df.to_excel(OUTPUT_FILE, index=False)
print(f"\n✅ Done. Saved to {OUTPUT_FILE}")


📄 Tweet 1 of 750
🏷️ Classification: Not relevant

📄 Tweet 2 of 750
🏷️ Classification: Not relevant

📄 Tweet 3 of 750
🏷️ Classification: Ageist

📄 Tweet 4 of 750
🏷️ Classification: Not relevant

📄 Tweet 5 of 750
🏷️ Classification: Not relevant

📄 Tweet 6 of 750
🏷️ Classification: Not relevant

📄 Tweet 7 of 750
🏷️ Classification: Not relevant

📄 Tweet 8 of 750
🏷️ Classification: Ageist

📄 Tweet 9 of 750
🏷️ Classification: Not relevant

📄 Tweet 10 of 750
🏷️ Classification: Not relevant

📄 Tweet 11 of 750
🏷️ Classification: Age-positive

📄 Tweet 12 of 750
🏷️ Classification: Ageist

📄 Tweet 13 of 750
🏷️ Classification: Age-positive

📄 Tweet 14 of 750
🏷️ Classification: Not relevant

📄 Tweet 15 of 750
🏷️ Classification: Not relevant

📄 Tweet 16 of 750
🏷️ Classification: Not relevant

📄 Tweet 17 of 750
🏷️ Classification: Not relevant

📄 Tweet 18 of 750
🏷️ Classification: Age-positive

📄 Tweet 19 of 750
🏷️ Classification: Not relevant

📄 Tweet 20 of 750
🏷️ Classification: Not relevant

📄 Twee

In [ ]:
pred_df = pd.read_excel("Gemma3_old_prompt1.xlsx")
ans_df  = pd.read_excel("750-tweets-answer.xlsx")

# Column names
pred_col = "classification"
aging_col = "aging related (yes/ no)"
ageist_col = "ageist (yes/ no)"
pos_col = "age positive (yes/ no)"

# -----------------------------
# Step 1: Create gold label using your rules
# -----------------------------
def create_gold_label(row):
    aging = row[aging_col].strip().upper()
    ageist = row[ageist_col].strip().upper()
    agepos = row[pos_col].strip().upper()

    if aging == "Y" and ageist == "Y":
        return "Ageist"
    elif aging == "Y" and agepos == "Y":
        return "Age-positive"
    else:
        return "Not relevant"

ans_df["gold_label"] = ans_df.apply(create_gold_label, axis=1)

# -----------------------------
# Step 2: Clean predicted label formatting
# -----------------------------
pred_df["pred_label"] = (
    pred_df[pred_col]
    .astype(str)
    .str.strip()
    .str.replace("-", "")       # e.g., "Age-positive" vs "Age positive"
    .str.title()                # normalize casing
)

# Normalize known values
pred_df["pred_label"] = pred_df["pred_label"].replace({
    "Agepositive": "Age-positive",
    "Ageist": "Ageist",
    "Not Relevant": "Not relevant",
})

# -----------------------------
# Step 3: Compare predicted label vs. gold label
# -----------------------------
gold = ans_df["gold_label"].to_numpy()
pred = pred_df["pred_label"].to_numpy()

pred_df["correct"] = pred == gold

accuracy = pred_df["correct"].mean()

print("\n====================================")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct predictions: {pred_df['correct'].sum()} / {len(pred_df)}")
print("====================================\n")

# Save output
pred_df.to_excel("Gemma3_prmp1_accuracy_output.xlsx", index=False)